In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer

# === STEP 1: Paths ===
base_path = "/kaggle/input/cotton/dataset"
train_dir = os.path.join(base_path, "cotton disease/train")
test_dir = os.path.join(base_path, "cotton disease/test")
predict_dir = os.path.join(base_path, "predictions")
pesticide_csv = os.path.join(base_path, "cotton_pesticide.csv")

# === STEP 2: Load Data ===
img_height, img_width = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

val_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

# Print classes for debug
class_labels = list(train_gen.class_indices.keys())
print("Class Labels:", class_labels)

# === STEP 3: Build Simple CNN ===
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(class_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# === STEP 4: Train Model ===
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

history = model.fit(train_gen, epochs=10, validation_data=val_gen, callbacks=callbacks)

# === STEP 5: Load Pesticide Dataset ===
pesticide_df = pd.read_csv(pesticide_csv)
pesticide_df.columns = [col.strip().lower().replace("\n", " ").replace("  ", " ") for col in pesticide_df.columns]
pesticide_df['disease'] = pesticide_df['disease'].str.strip().str.lower()

# === STEP 6: Pesticide Recommendation Function ===
def recommend_pesticide(disease_name):
    disease_name = disease_name.lower().strip()
    match = pesticide_df[pesticide_df['disease'] == disease_name]
    if not match.empty:
        row = match.iloc[0]
        return {
            "Description": row.get('description', 'N/A'),
            "Pesticide (Small Region)": row.get('pesticide (small region)', 'N/A'),
            "Dosage (Small Region)": row.get('dosage (small region)', 'N/A'),
            "Pesticide (Large Region)": row.get('pesticide (large region)', 'N/A'),
            "Dosage (Large Region)": row.get('dosage (large region)', 'N/A'),
            "Organic Method": row.get('organic method', 'N/A')
        }
    else:
        return {"Note": "No pesticide data found for this disease."}

# === STEP 7: Predict + Recommend Function ===
def predict_and_recommend(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_index]

    print("Predicted Disease:", predicted_class)
    recommendation = recommend_pesticide(predicted_class)
    print("Pesticide Recommendation:")
    for k, v in recommendation.items():
        print(f"{k}: {v}")

# === STEP 8: Try an Example ===
predict_and_recommend(os.path.join(predict_dir, "cotton_d2.jpg"))



2025-04-22 06:49:49.644744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745304590.082616      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745304590.196293      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 8640 images belonging to 6 classes.
Found 721 images belonging to 6 classes.
Class Labels: ['Aphids', 'Army_worm', 'Bacterial_Blight', 'Healthy', 'Powdery_Mildew', 'Target_spot']


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745304616.060453      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745304616.061187      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       3,686,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1745304623.757753     109 service.cc:148] XLA service 0x7b00d800c190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745304623.759621     109 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745304623.759641     109 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745304624.104922     109 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/270 ━━━━━━━━━━━━━━━━━━━━ 41:12 9s/step - accuracy: 0.1250 - loss: 1.7701

I0000 00:00:1745304627.991484     109 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


270/270 ━━━━━━━━━━━━━━━━━━━━ 117s 402ms/step - accuracy: 0.3007 - loss: 1.7065 - val_accuracy: 0.6713 - val_loss: 0.9310
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - accuracy: 0.6209 - loss: 1.0325 - val_accuracy: 0.7448 - val_loss: 0.7398
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - accuracy: 0.7188 - loss: 0.7805 - val_accuracy: 0.8128 - val_loss: 0.5344
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 115ms/step - accuracy: 0.7885 - loss: 0.5975 - val_accuracy: 0.8336 - val_loss: 0.5069
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 31s 114ms/step - accuracy: 0.8519 - loss: 0.4262 - val_accuracy: 0.8571 - val_loss: 0.4128
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 114ms/step - accuracy: 0.8945 - loss: 0.3172 - val_accuracy: 0.8544 - val_loss: 0.4347
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 114ms/step - accuracy: 0.9288 - loss: 0.2117 - val_accuracy: 0.8544 - val_loss: 0.4397
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 32s 115ms/step - accuracy: 0.9427 - loss: 0.1763 - va

In [2]:
# Updated pesticide recommendation function with normalization
def normalize(text):
    return text.strip().lower().replace('_', ' ')

def recommend_pesticide(disease_name):
    disease_name = normalize(disease_name)
    pesticide_df['normalized_disease'] = pesticide_df['disease'].apply(normalize)
    
    match = pesticide_df[pesticide_df['normalized_disease'] == disease_name]
    if not match.empty:
        row = match.iloc[0]
        return {
            "Description": row.get('description', 'N/A'),
            "Pesticide (Small Region)": row.get('pesticide (small region)', 'N/A'),
            "Dosage (Small Region)": row.get('dosage (small region)', 'N/A'),
            "Pesticide (Large Region)": row.get('pesticide (large region)', 'N/A'),
            "Dosage (Large Region)": row.get('dosage (large region)', 'N/A'),
            "Organic Method": row.get('organic method', 'N/A')
        }
    else:
        return {"Note": "No pesticide data found for this disease."}


In [3]:
predict_and_recommend(os.path.join(predict_dir, "cotton_d2.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Disease: Target_spot
Pesticide Recommendation:
Description: Fungal disease caused by 
Corynespora cassiicola
Pesticide (Small Region):  Mancozeb 75% WP
Dosage (Small Region):  1kg/ha
Pesticide (Large Region):  Azoxystrobin 23% SC
Dosage (Large Region): 500ml/ha
Organic Method: *Destroy infected plant
*Rotate crops 
